In [1]:
from pathlib import Path
import math
import shutil
from copy import deepcopy

from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter


In [2]:
tate_podcast_ID = "vsp69jYlYsg"
tate_short_clip_ID = "Tcoped8NLpE"
I_like_turtles_ID = "CMNry4PE93Y"
tucker_speech_ID = "N32UPXGChgo"
fireship_vid_ID = "PR_ykicOZYU"
mkbhd_short_ID = "KG6Nu72Avqg"

transcript = YouTubeTranscriptApi.get_transcript(tate_short_clip_ID)


In [3]:
def format_transcript(transcript: list):
    formatted_transcript = ""

    for entry in transcript:
        entry_text = entry["text"]
        entry_start = entry["start"]
        entry_string = f"{entry_start}|{entry_text};\n"
        formatted_transcript += entry_string

    return formatted_transcript


formatted_transcript = format_transcript(transcript)

# with open('formatted_transcript.txt', 'w') as the_file:
#     the_file.write(formatted_transcript)

formatted_transcript


"13.12|that's the first thing the second thing;\n14.799|is you need to be perspicacious you need;\n16.72|to understand that in this world there's;\n18.8|a whole bunch of people doing amazing;\n20.24| that you are not doing and that;\n22.24|needs to piss you off because it pisses;\n24.56|you off you become motivated all of a;\n26.48|sudden i was the only one who was pissed;\n28.56|off when that aston martin was next to;\n30.48|me and i had my sales job i was the only;\n32.239|one who was pissed off when i saw that;\n33.52|ferrari drive past other people were not;\n35.36|annoyed by it and they're not annoyed by;\n36.8|it they're not driven to beat it you;\n38.879|understand they're not driven to be;\n40.0|where that person was you need to be;\n42.0|annoyed and i'm telling you i'm telling;\n43.92|you to be angry;\n45.36|anger is a fantastic force;\n47.68|like i said the world tells you not to;\n49.039|be arrogant tells you not to be angry;\n50.48|it's two things it tells you to do don't;\

The below code should output all Bing GPT prompts to prompts/ in format prompt_#
Prompts are being tested with "More precise" version of Bing GPT

In [4]:
# specify prompts dir
prompts_dir = Path("prompts/")

# remove prompts dir if it already exists
if prompts_dir.exists():
    shutil.rmtree(prompts_dir)

# create prompts dir
prompts_dir.mkdir(exist_ok=True, parents=True)

# number to keep track of prompt number
prompt_number = 0


In [5]:
# create prompts for accepting formatted transcript

# maximum amount of character input left after "I want to share..."
max_part_chars = 1780

# get number of necessary prompts for accepting transcript
num_transcript_prompts = math.ceil(len(formatted_transcript) / max_part_chars)

transcript_pieces = formatted_transcript.split("\n")

transcript_peice_num = 0
for transcript_prompt_num in range(num_transcript_prompts):

    transcript_prompt = f"""I want to share part {transcript_prompt_num+1} of a formatted transcript.
The transcript entries are formatted <entry_timestamp>|<entry_text>;.
Please only respond with UNDERSTOOD to indicate that you have recieved the formatted transcript.
"""

    # compile peices of transcript prompt such that no entry is left off 
    while len(transcript_prompt) < max_part_chars:
        if transcript_peice_num == len(transcript_pieces):
            break

        transcript_prompt += transcript_pieces[transcript_peice_num]
        transcript_prompt += "\n"
        transcript_peice_num += 1
    
    # save prompt as text file
    prompt_save_path = Path(f"{prompts_dir}/prompt_{prompt_number}.txt")
    with open(prompt_save_path, 'w') as prompt_file:
        prompt_file.write(transcript_prompt)

    prompt_number += 1

In [6]:
# create compilation prompt

compilation_prompt = """Please automatically convert these formatted transcripts into well-punctated text. 
The result should be a well-punctuated video transcript."""

prompt_save_path = Path(f"{prompts_dir}/prompt_{prompt_number}.txt")
with open(prompt_save_path, 'w') as prompt_file:
    prompt_file.write(compilation_prompt)

prompt_number += 1

In [7]:
# create prompt to find a certain clip


clip_category = "interesting"
clip_length = 45

compilation_prompt = f"""Given the well-punctated text and the formatted transcripts, find a {clip_category} clip that is about {clip_length} seconds. 
Make sure that this clip could be contained without any other context and does not get cut off without the speaker finishing a thought.
In other words, if someone only heard the speaker during this clip, they would not be confused by where the clip is beginning or feel that it ended without a conclusion.
Make sure that this clip would create a {clip_category} YouTube short or Tik Tok video.
Give the beginning and end time stamps of this clip based on the formatted transcript.
Use the beginning and end time stamps to make sure this clip is about {clip_length} seconds."""

prompt_save_path = Path(f"{prompts_dir}/prompt_{prompt_number}.txt")
with open(prompt_save_path, 'w') as prompt_file:
    prompt_file.write(compilation_prompt)

prompt_number += 1